#Importing packages

In [ ]:
import pandas as pd 
import cv2
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from matplotlib import patches

#import h5py    
import numpy as np  
import pandas as pd

from scipy.optimize import linear_sum_assignment
#! pip install mpmath
from mpmath import *
import datetime as dt
import json 
import copy

import os 
import time 
import plotly.express as px
import plotly.graph_objects as go
import motmetrics as mm
import numpy as np
import re
os.chdir("../evaluation")
!pwd

In [ ]:

label_file= "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/labels_with_atq.json"
track_base = "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/results/"

def read_data(file, type_file='track'):
    #here we will go through detections of deepsort 
    import json
    track={}
    with open(file) as f:
        json_file = json.load(f) 

    for frame, detections in json_file.items():
        frame=int(frame)
        track[frame]={}
        for id, detection in detections.items():
            track[frame][id]={}
            track[frame][id]["rectangle"]= tuple(detection)

    return track


label_track = read_data(label_file, type_file="label")



def iou (boxA,boxB):
    boxA=[boxA[0],boxA[1],boxA[0]+boxA[2],boxA[1]+boxA[3]]
    boxB=[boxB[0],boxA[1],boxB[0]+boxB[2],boxB[1]+boxB[3]]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1))
    boxBArea = abs((boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1))
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou  #np.linalg.norm(np.array([float(track[0]), float(track[1])+float(track[3])/2])-np.array([600,17.5]))




def precise_accuracy_track(label_track, model_track, basic_tracker=False):
    """cette fonction calcule le f1-score recall, accuracy des model par rapport au background
    dedans, les score des trackers et des hMM based tracker sont calculés différemment car quand le hmm based tracker est seuillé, 
    il y'a des id de track qu'il ne retourne pas dans son fichier de resultat.

    Args:
        label_track (_type_): _description_
        model_track (_type_): _description_
        basic_tracker (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    """
    start=True 
    nbr_frame=0
    acc =0
    rec=0
    """if basic_tracker==True:
        matching={}
        for frame_id in label_track.keys():
            for label_atq, label_box in label_track[frame_id].items() : 
                if label_atq not in matching.keys() and label_atq!="observed":
                    matching[label_atq]=None 
                    break"""
            
    
    def match_track_and_atq(label_track, model_track):
            matching={}
            label_track_only_concerned = {}
            for frame_id in label_track.keys():
                label_track_only_concerned[frame_id]= {}
                if frame_id in model_track.keys() :
                    for label_atq, label_box in label_track[frame_id].items() : 
                        if float(label_atq)>4800:
                            label_track_only_concerned[frame_id][label_atq] = label_track[frame_id][label_atq]
                            
                        if label_atq not in matching.keys():
                            max_iou=float('-inf')
                            for model_atq, model_box in model_track[frame_id].items(): 
                                #print(model_atq)
                                if  label_atq!="observed" and model_atq!="observed":#fix the problem with the obseved on the label 
                                    
                                    """if str(model_atq)==str(label_atq):
                                        print(model_atq, model_box["rectangle"], label_box["rectangle"])
                                    """
                                    
                                    tmp = iou(model_box["rectangle"], label_box["rectangle"])
                                    if tmp>=max_iou:
                                        matching [label_atq]=model_atq
                                        max_iou =tmp
                
                if len(matching.keys())== len(label_track[frame_id].keys()):
                    break
            return label_track_only_concerned , {value:key for key, value in matching.items() if float(key)>4800}
    if basic_tracker==True:
        label_tracki, matching = match_track_and_atq(label_track, model_track)
    else:
        matching= {k:k for k in model_track[1].keys() if  'identities' not in k}
    for frame_id in label_track.keys() :
        if frame_id in model_track.keys() :
            nbr_frame+=1
            matching_frame={}
            for model_atq, model_box in model_track[frame_id].items() :
                    max_iou=float('-inf')
                    atq_matching_model =None
                    for  label_atq, label_box in label_track[frame_id].items():
                        if  label_atq!="observed" and model_atq!="observed" :#fix the problem with the obseved on the label 
                            tmp = iou(model_box["rectangle"], label_box["rectangle"])
                            if tmp>max_iou:
                                max_iou =tmp
                                atq_matching_model = label_atq
                                
                    #if basic_tracker==True:
                    if model_atq in matching.keys(): #ca c'est pour les modèles qui crèent trop de nouvelles identités
                            matching_frame[matching[model_atq] ]=atq_matching_model 
                    """else:
                        matching_frame[model_atq ]=atq_matching_model """
                                
            filtered ={key:value for key,value in matching_frame.items() if value==key }
            if len(matching_frame.keys())!=0:
                acc = acc + len(filtered.keys())/ len(matching_frame.keys())
                rec = rec+ len(filtered.keys())/ len(matching.keys())
                if len(filtered.keys())/ len(label_track[frame_id].keys())>1:
                    print("stop")
        else:
            print("weird thing")

    
    acc = acc/nbr_frame
    rec=rec/nbr_frame
    f1=2*acc*rec/(acc+rec)
    print("ok", acc, rec) 
    return acc  , rec , f1          


tag = "_feeder_H_0.07.json"
observation_file="/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/GR77_20200512_111314DBN_result_with_observations"+tag
Hmm_result_file="/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/GR77_20200512_111314_with_atq_tracking_with_HMM_result"+tag
    
hmm_track = read_data(Hmm_result_file)
acc, rec, f1= precise_accuracy_track(label_track, hmm_track, basic_tracker=False)
#acc, rec, f1= precise_accuracy_track(label_track, hmm_track, basic_tracker=True)
new_row= {'nbr of visits':"feeder", 'accuracy':acc, 'recall':rec, "f1":f1}

#Util functions

In [ ]:

def find_rectangle (pts,image_file=None):
    """
    this function provide straigth rectangle of a  set of points
    """
    pts = pts[~np.isnan(pts).any(axis=1)]
    pts= pts.astype(int)
    #print(pts)

    x,y,w,h = cv2.boundingRect(pts)
    if image_file!=None:
        try:
            image = mpimg.imread(image_file)
            fig, ax = plt.subplots(1,figsize=(20,15))
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none') # Coordinates: (left, top), width, height
            ax.add_patch(rect)
            ax.scatter(pts[:, 0],pts[:,1], marker="o", color="red", s=2)
            ax.scatter(x+w/2,y+h/2, marker="o", color="yellow", s=2)
            ax.imshow(image)
            plt.show()
        except:
            print("Une erreur durant l'affichage du rectange droit minimal")

    return (x,y,w,h)

#Test the find_rectangle fonction
"""pts=np.array([[785,43],[798,78],[827,48],[833,69],[947,132]])
image_file="pig_tracking_dataset/labeled-data/GR77_20200512_111314/img00000.png"
find_rectangle(pts,image_file=image_file)"""

#keep data  in dictionnary of image of individual of body part: function dlc_data_to_dct
#labels=pd.read_csv("pig_tracking_dataset/labeled-data/GR77_20200512_111314/CollectedData_CDPQ_experiment.csv")

def read_data(file, type_file='track'):
    #here we will go through detections of deepsort 
    import json
    track={}
    with open(file) as f:
        json_file = json.load(f) 

    for frame, detections in json_file.items():
        frame=int(frame)
        track[frame]={}
        for id, detection in detections.items():
            track[frame][id]={}
            track[frame][id]["rectangle"]= tuple(detection)

    return track

def iou (boxA,boxB):
    boxA=[boxA[0],boxA[1],boxA[0]+boxA[2],boxA[1]+boxA[3]]
    boxB=[boxB[0],boxA[1],boxB[0]+boxB[2],boxB[1]+boxB[3]]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1))
    boxBArea = abs((boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1))
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou  #np.linalg.norm(np.array([float(track[0]), float(track[1])+float(track[3])/2])-np.array([600,17.5]))

"""
Paramètres de l'évaluation:
- le max_iou: fiixé à 0.85 soit on accepte les  iou de 15%
"""
################################################################################################
def evaluate (tracks,track_result, limit=float("inf")) :
    acc = mm.MOTAccumulator(auto_id=False)
    for key, track in tracks.items():
        if key in  track_result.keys()  :
            frame_detection=track_result[key]
            real_animals=np.array([list(individual["rectangle"]) for key, individual  in list(track.items()) if float(key)>=4800])
            hypothesis_animal= np.array([list(individual["rectangle"]) for key,individual  in list(frame_detection.items()) if 'identities' not in key])
            #print(len(real_animals), "*****", len(hypothesis_animal))
            distances = mm.distances.iou_matrix(real_animals, hypothesis_animal, max_iou=0.75)
            #print("deepsort",hypothesis_animal)
            #print("real",real_animals)
            #print(distances)
            #print(real_animals)
            #print([ key1 for key1 in frame_detection.keys()])
            acc.update([float(key1) for key1 in list(track.keys()) if float(key1)>=4800],[ float(re.search(r'^\d+', key1)[0]) for key1 in frame_detection.keys() if 'identities' not in key1],distances, frameid=int(key))
        if int(key)>= limit:
              break
    return acc 


#acc_deepsort= evaluate(tracks,deepsort_track)
#acc_deeplabcut=evaluate(tracks, deeplabcut_track)
def Evaluate_over_time(tracks, deepsort_track,column_indice): 
    accs_deepsort=[]
    frames=[]
    prev=0
    for key, track in tracks.items():
        if key-prev>99:
            acc=evaluate(tracks,deepsort_track,limit=int(key))
            accs_deepsort.append(acc)
            frames.append(int(key))
            prev=int(key)
            
    mh = mm.metrics.create()
    summary = mh.compute_many(
        accs_deepsort,
        #[acc_deepsort, acc_deeplabcut],
        #metrics=mm.metrics.motchallenge_metrics,
        #metrics=['idf1','num_frames', 'mota', 'motp'],
        names=frames, #['deepsort','deeplabcut'],
        generate_overall=True
        )

    strsummary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names
    )
    print("cumulative summary")
    print(strsummary)
    for col in summary.columns:
        summary.columns = summary.columns.str.replace(col, col+'_'+column_indice)
    return summary

def Evaluate_single_frames_over_time(tracks, deepsort_track,column_indice): 
    ""
    accs_deepsort=[]
    frames=[]
    for key, track in tracks.items():
        #############################################to arrange for other datasets
        acc=evaluate({ 16:tracks[16], 0:tracks[0],key:tracks[key]},deepsort_track,limit=int(2))
        accs_deepsort.append(acc)
        frames.append(int(key))
        
    mh = mm.metrics.create()
    summary = mh.compute_many(
        accs_deepsort,
        #[acc_deepsort, acc_deeplabcut],
        #metrics=mm.metrics.motchallenge_metrics,
        #metrics=['idf1','num_frames', 'mota', 'motp'],
        names=frames, #['deepsort','deeplabcut'],
        generate_overall=True)
    strsummary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names)
    print("summary per frame")
    #print(summary)
    print(strsummary)
    for col in summary.columns:
        summary.columns = summary.columns.str.replace(col, col+'_'+column_indice)
    
    return summary

def precise_accuracy_track2(label_track, model_track, basic_tracker=False):
    """cette fonction calcule le f1-score recall, accuracy des model par rapport au background
    dedans, les score des trackers et des hMM based tracker sont different car le hmmbased tracker a les vrais identités donc facilite les calculs 

    il y'a des id de track qu'il ne retourne pas dans son fichier de resultat.

    Args:
        label_track (_type_): _description_
        model_track (_type_): _description_
        basic_tracker (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    """
    import re
    start=True 
    nbr_frame=0
    acc =0
    rec=0
    accs=[]
    f1s=[]
    recs=[]
    indices=[]
    
    accs_ind=[]
    f1s_ind=[]
    recs_ind=[]
    """if basic_tracker==True:
        matching={}
        for frame_id in label_track.keys():
            for label_atq, label_box in label_track[frame_id].items() : 
                if label_atq not in matching.keys() and label_atq!="observed":
                    matching[label_atq]=None 
                    break"""
            
    
    def match_track_and_atq(label_track, model_track):
            """
            retourne au format label_key:model_key les correspondances entre les identités du model et nos label et ceci pour chaque label identity
            """
            matching={}
            for frame_id in label_track.keys():
                if frame_id in model_track.keys() :
                    
                    for label_atq, label_box in label_track[frame_id].items() : 
                        if label_atq not in matching.keys():
                            max_iou=float('-inf')
                            for model_atq, model_box in model_track[frame_id].items(): 
                                #print(model_atq)
                                if  label_atq!="observed" and model_atq!="observed":#fix the problem with the obseved on the label 
                                    
                                    """if str(model_atq)==str(label_atq):
                                        print(model_atq, model_box["rectangle"], label_box["rectangle"])
                                    """
                                    
                                    tmp = iou(model_box["rectangle"], label_box["rectangle"])
                                    if tmp>=max_iou:
                                        matching [label_atq]=model_atq
                                        max_iou =tmp
                
                if len(matching.keys())== len(label_track[frame_id].keys()):
                    break
            return {value:key for key, value in matching.items() if float(key)>4800} #why if float(key)>4800? We have this specific condition because we want to access the basic tracker on the same base with the tracker so only on identities the tracker also have
    
    if basic_tracker==True:
        matching = match_track_and_atq(label_track, model_track)
        print(matching)
    else:
        matching= {k:k for k in model_track[1].keys() if  'identities' not in k}

    for frame_id in label_track.keys() :
        if frame_id in model_track.keys() :
            nbr_frame+=1
            if frame_id==10066:
                print("stop")
            matching_frame={}
            for model_atq, model_box in model_track[frame_id].items() :
                    max_iou=float('-inf')
                    for  label_atq, label_box in label_track[frame_id].items():
                        if  label_atq!="observed" and model_atq!="observed" :#fix the problem with the obseved on the label 
                            tmp = iou(model_box["rectangle"], label_box["rectangle"])
                            if tmp>max_iou:
                                max_iou =tmp
                                atq_matching_model = label_atq
                                
                    #if basic_tracker==True:
                    if model_atq in matching.keys(): #ca c'est pour les modèles qui crèent trop de nouvelles identités
                            matching_frame[matching[model_atq] ]=atq_matching_model 
                    """else:
                        matching_frame[model_atq ]=atq_matching_model """
                                
            filtered ={key:value for key,value in matching_frame.items() if value==key }
            if len(matching_frame.keys())!=0:
                acc = acc + len(filtered.keys())/ len(matching_frame.keys())
                rec = rec+ len(filtered.keys())/ len(label_track[frame_id].keys())
                if len(filtered.keys())/ len(label_track[frame_id].keys())>1:
                    print("stop")
                #accs.append(acc)
                #recs.append(rec)
                accs.append(acc/nbr_frame)
                recs.append(rec/nbr_frame)
                if acc!=0.0 and recs!=0:
                    f1=2*(acc/nbr_frame)*(rec/nbr_frame)/((acc/nbr_frame)+(rec/nbr_frame))
                else:
                    f1=0.0
                #f1s.append(f1)
                f1s.append(f1)
                indices.append(int(frame_id))
        else:
            if nbr_frame!=0:
                accs.append(acc/nbr_frame)
                recs.append(rec/nbr_frame)
                if acc!=0.0 and recs!=0:
                    f1=2*(acc/nbr_frame)*(rec/nbr_frame)/((acc/nbr_frame)+(rec/nbr_frame))
                else:
                    f1=0.0

                #f1s.append(f1)
                f1s.append(f1)
                indices.append(int(frame_id))
            
    acc = acc/nbr_frame
    rec=rec/nbr_frame
    if acc!=0 and rec!=0:
        f1=2*acc*rec/(acc+rec)
    else: 
        f1=0
    print(acc, rec,f1) 
    return accs  , recs , f1s, indices        

def precise_accuracy_track(label_track, model_track, basic_tracker=False):
    """cette fonction calcule le f1-score recall, accuracy des model par rapport au background
    dedans, les score des trackers et des hMM based tracker sont calculés différemment car quand le hmm based tracker est seuillé, 
    il y'a des id de track qu'il ne retourne pas dans son fichier de resultat.

    Args:
        label_track (_type_): _description_
        model_track (_type_): _description_
        basic_tracker (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    """
    import re
    start=True 
    nbr_frame=0
    acc =0
    rec=0
    nbr_labeled_frame=0
    
  
    accs=[]
    f1s=[]
    recs=[]
    indices=[]
    """if basic_tracker==True:
        matching={}
        for frame_id in label_track.keys():
            for label_atq, label_box in label_track[frame_id].items() : 
                if label_atq not in matching.keys() and label_atq!="observed":
                    matching[label_atq]=None 
                    break"""
            
    
    def match_track_and_atq(label_track, model_track):
            matching={}
            label_track_only_concerned = {}
            for frame_id in label_track.keys():
                label_track_only_concerned[frame_id]= {}
                if frame_id in model_track.keys() :
                    for label_atq, label_box in label_track[frame_id].items() : 
                        if float(label_atq)>4800:
                            label_track_only_concerned[frame_id][label_atq] = label_track[frame_id][label_atq]
                            
                        if label_atq not in matching.keys():
                            max_iou=float('-inf')
                            for model_atq, model_box in model_track[frame_id].items(): 
                                #print(model_atq)
                                if  label_atq!="observed" and model_atq!="observed":#fix the problem with the obseved on the label 
                                    
                                    """if str(model_atq)==str(label_atq):
                                        print(model_atq, model_box["rectangle"], label_box["rectangle"])
                                    """
                                    
                                    tmp = iou(model_box["rectangle"], label_box["rectangle"])
                                    if tmp>=max_iou:
                                        matching [label_atq]=model_atq
                                        max_iou =tmp
                
                if len(matching.keys())== len(label_track[frame_id].keys()):
                    break
            return label_track_only_concerned , {value:key for key, value in matching.items() if float(key)>4800}
    if basic_tracker==True:
        label_tracki, matching = match_track_and_atq(label_track, model_track)
    else:
        matching= {k:k for k in model_track[1].keys() if  'identities' not in k}
    for frame_id in model_track.keys() :
        nbr_frame+=1
        if frame_id in label_track.keys() :
            nbr_labeled_frame+=1
            matching_frame={}
            for model_atq, model_box in model_track[frame_id].items() :
                    max_iou=float('-inf')
                    atq_matching_model =None
                    for  label_atq, label_box in label_track[frame_id].items():
                        if  label_atq!="observed" and model_atq!="observed" :#fix the problem with the obseved on the label 
                            tmp = iou(model_box["rectangle"], label_box["rectangle"])
                            if tmp>max_iou:
                                max_iou =tmp
                                atq_matching_model = label_atq
                                
                    #if basic_tracker==True:
                    if model_atq in matching.keys(): #ca c'est pour les modèles qui crèent trop de nouvelles identités
                            matching_frame[matching[model_atq] ]=atq_matching_model 
                    """else:
                        matching_frame[model_atq ]=atq_matching_model """
                                
            filtered ={key:value for key,value in matching_frame.items() if value==key }
            if len(matching_frame.keys())!=0:
                acc = acc + len(filtered.keys())/ len(matching_frame.keys())
                rec = rec+ len(filtered.keys())/ len(matching.keys())
                if len(filtered.keys())/ len(label_track[frame_id].keys())>1:
                    print("stop")
                accs.append(acc/nbr_labeled_frame)
                recs.append(rec/nbr_labeled_frame)
                indices.append(nbr_frame)
                if acc+rec!=0:
                    f1s.append((2*acc*rec/(acc+rec))/nbr_labeled_frame )
                else:
                    f1s.append(0)
        """else:
            print("weird thing")"""

    
    acc = acc/nbr_labeled_frame
    rec=rec/nbr_labeled_frame
    f1=2*acc*rec/(acc+rec)
    print(acc, rec) 
    return accs  , recs , f1s, indices            
               
def precise_accuracy_track_per_frame(label_track, model_track, basic_tracker=False):
    """cette fonction calcule le f1-score recall, accuracy des model par rapport au background
    dedans, les score des trackers et des hMM based tracker sont calculés différemment car quand le hmm based tracker est seuillé, 
    il y'a des id de track qu'il ne retourne pas dans son fichier de resultat.

    Args:
        label_track (_type_): _description_
        model_track (_type_): _description_
        basic_tracker (bool, optional): _description_. Defaults to False.

    Returns:
        _type_: _description_
    """
    import re
    start=True 
    nbr_frame=0
    acc =0
    rec=0
    nbr_labeled_frame=0
    
  
    accs=[]
    f1s=[]
    recs=[]
    indices=[]
    """if basic_tracker==True:
        matching={}
        for frame_id in label_track.keys():
            for label_atq, label_box in label_track[frame_id].items() : 
                if label_atq not in matching.keys() and label_atq!="observed":
                    matching[label_atq]=None 
                    break"""
            
    
    def match_track_and_atq(label_track, model_track):
            matching={}
            label_track_only_concerned = {}
            for frame_id in label_track.keys():
                label_track_only_concerned[frame_id]= {}
                if frame_id in model_track.keys() :
                    for label_atq, label_box in label_track[frame_id].items() : 
                        if float(label_atq)>4800:
                            label_track_only_concerned[frame_id][label_atq] = label_track[frame_id][label_atq]
                            
                        if label_atq not in matching.keys():
                            max_iou=float('-inf')
                            for model_atq, model_box in model_track[frame_id].items(): 
                                #print(model_atq)
                                if  label_atq!="observed" and model_atq!="observed":#fix the problem with the obseved on the label 
                                    
                                    """if str(model_atq)==str(label_atq):
                                        print(model_atq, model_box["rectangle"], label_box["rectangle"])
                                    """
                                    
                                    tmp = iou(model_box["rectangle"], label_box["rectangle"])
                                    if tmp>=max_iou:
                                        matching [label_atq]=model_atq
                                        max_iou =tmp
                
                if len(matching.keys())== len(label_track[frame_id].keys()):
                    break
            return label_track_only_concerned , {value:key for key, value in matching.items() if float(key)>4800}
    if basic_tracker==True:
        label_tracki, matching = match_track_and_atq(label_track, model_track)
    else:
        matching= {k:k for k in model_track[1].keys() if  'identities' not in k}
    for frame_id in model_track.keys() :
        nbr_frame+=1
        if frame_id in label_track.keys() :
            nbr_labeled_frame+=1
            matching_frame={}
            for model_atq, model_box in model_track[frame_id].items() :
                    max_iou=float('-inf')
                    atq_matching_model =None
                    for  label_atq, label_box in label_track[frame_id].items():
                        if  label_atq!="observed" and model_atq!="observed" :#fix the problem with the obseved on the label 
                            tmp = iou(model_box["rectangle"], label_box["rectangle"])
                            if tmp>max_iou:
                                max_iou =tmp
                                atq_matching_model = label_atq
                                
                    #if basic_tracker==True:
                    if model_atq in matching.keys(): #ca c'est pour les modèles qui crèent trop de nouvelles identités
                            matching_frame[matching[model_atq] ]=atq_matching_model 
                    """else:
                        matching_frame[model_atq ]=atq_matching_model """
                                
            filtered ={key:value for key,value in matching_frame.items() if value==key }
            if len(matching_frame.keys())!=0:
                acc =  len(filtered.keys())/ len(matching_frame.keys())
                rec =  len(filtered.keys())/ len(matching.keys())
                if len(filtered.keys())/ len(label_track[frame_id].keys())>1:
                    print("stop")
                accs.append(acc)
                recs.append(rec)
                indices.append(nbr_frame)
                if acc+rec!=0:
                    f1s.append((2*acc*rec/(acc+rec)) )
                else:
                    f1s.append(0)
        """else:
            print("weird thing")"""

    
    """acc = acc/nbr_labeled_frame
    rec=rec/nbr_labeled_frame
    f1=2*acc*rec/(acc+rec)
    print(acc, rec) """
    return accs  , recs , f1s, indices            
               

In [18]:
label_file= "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/labels_with_atq.json"
re_id_file = "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/GR77_20200512_111314DBN_re_id.json"
hmm_file="/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder_H_0.07.json"
bytetrack_file="/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/GR77_20200512_111314tracking_result.json"

label_track = read_data(label_file)
hmm_track = read_data(hmm_file)
bytetrack = read_data(bytetrack_file)
re_id_track= read_data(re_id_file)
results_per_id={}
def replace_based_on_init(re_id_track, hmm=False):
    """get initial correspondances and replac them in the rest of the track 
    Args:
        re_id_track (_type_): _description_
    Returns:
        dct: track with replaced id
    """
    correspondances_re_id={}
    for frame_id, frame_infos in label_track.items():
        if frame_id in re_id_track.keys():
            for id_label, label_box in frame_infos.items():
                max_iou =0
                max_id=0
                for id_model, model_box in re_id_track[frame_id].items():
                    aiou = iou(label_box['rectangle'], model_box["rectangle"])
                    if aiou > max_iou:
                        max_id = id_model
                        max_iou=aiou
                correspondances_re_id[id_label] = max_id
            break
    track = re_id_track
    crs = correspondances_re_id
    result_per_id = {val:[] for  val in crs.keys()}
    indices=[]
    if hmm==False:
        for frame_id, frame_infos in label_track.items():
            if frame_id in track.keys():
                indices.append(int(frame_id))
                for label_id, rectangle in frame_infos.items():
                    if crs[label_id] in  track[frame_id].keys():
                        a= iou(rectangle['rectangle'], track[frame_id][crs[label_id]]['rectangle'] ) 
                        if a>0:
                            a=1
                        result_per_id[label_id].append(a)
                    else: 
                        result_per_id[label_id].append(0.2)
    else:
        for frame_id, frame_infos in label_track.items():
            if frame_id in track.keys():
                indices.append(int(frame_id))
                for label_id, rectangle in frame_infos.items():
                    if label_id in  track[frame_id].keys():
                        a= iou(rectangle['rectangle'], track[frame_id][label_id]['rectangle'] ) 
                        if a>0:
                            a=1
                        result_per_id[label_id].append(a)
                    else: 
                        result_per_id[label_id].append(0.2)
        
    return result_per_id, indices       

re_id, indices_re=replace_based_on_init(re_id_track)
hmm, indices_h = replace_based_on_init(hmm_track, hmm=True)
bytetrack, indices_b= replace_based_on_init(bytetrack)

fig = go.Figure()
for idx, atq in enumerate(re_id.keys()):
    fig.add_trace(go.Scatter(x=indices_re, y= np.array(re_id[atq])+idx,
                                mode='lines',
                                name='re-identification'+str(atq)))
    """fig.add_trace(go.Scatter(x=indices_b, y= np.array(bytetrack[atq])+idx,
                                mode='lines',
                                name='recall of Bytetrack'+str(atq)))"""
    fig.add_trace(go.Scatter(x=indices_h, y= np.array(hmm[atq])+idx,
                                mode='lines',
                                name='re-identification'+str(atq)))
    break

fig.show()

Define global variable such as location of results

In [ ]:
import plotly.express as px
label_file= "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/labels_with_atq.json"
track_base = "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/results/"
label_track = read_data(label_file, type_file="label")
visits =pd.read_csv("/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/visualize/observations.csv")
visits["verif"]=np.zeros(len(visits))
visits["verif"][[1,3,7]]=1

#pointer les visites sur lee graphe , en rouge les mauvaise et vert les bonnes ou noir et blanc 
#visits

In [ ]:

fig_acc = go.Figure()
fig_rec = go.Figure()
fig_f1 = go.Figure()
df =pd.DataFrame()
dct={"ByteTrack":"Bytetrack.json",  "ByteTrack+HMM":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder_H_0.07.json", "ByteTrack+Re-ID": "GR77_20200512_111314DBN_re_id_2.json"}# "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 
for idx, method in enumerate(dct.keys()):
    track_result= read_data(track_base+dct[method]) #### a refaire
    
    if method == "ByteTrack+HMM":
        df[method+"accuracy"], df[method+"recall"], df[method+"f1-score"], indices = precise_accuracy_track_per_frame(label_track, track_result, basic_tracker=False)
    else:
        df[method+"accuracy"], df[method+"recall"], df[method+"f1-score"], indices = precise_accuracy_track_per_frame(label_track, track_result, basic_tracker=True)
        

    """print(method)
    df.index =indices
    print(visits)"""
    fig_rec.add_trace(go.Scatter(x=df.index.values, y=df[method+"recall"].values,
                            mode='lines',
                            name='recall of '+method))
    
    fig_f1.add_trace(go.Scatter(x=df.index.values, y=df[method+"f1-score"].values, 
                            mode='lines',
                            name='f1-score of '+method))
    
    """fig_f1.add_trace(go.Scatter(x=df.index.values, y=df[method+"f1-score"].values, 
                            mode='markers',
                            name=''))"""
    
    fig_acc.add_trace(go.Scatter(x=df.index.values, y=df[method+"accuracy"].values,
                            mode='lines',
                            name='accuracy of '+method))
    
    
    df =pd.DataFrame()
        

fig_f1.update_layout(
            xaxis_title=dict(text="frame number"),
            yaxis_title=dict(text= "f1-score"),
    font=dict(
             #   color="black",
                size=12
            ),
            )
fig_acc.update_layout(
            xaxis_title="frame number",
            yaxis_title="accuracy",
    font=dict(
             #   color="black",
                size=12
            ),
            )
fig_rec.update_layout(
            xaxis_title="frame number",
            yaxis_title="recall",
    font=dict(
             #   color="black",
                size=12
            ),
            )

fig_f1.show()
fig_rec.show()
fig_acc.show()

##plot accuracy, recall and f1-score over time

In [ ]:

fig_acc = go.Figure()
fig_rec = go.Figure()
fig_f1 = go.Figure()
df =pd.DataFrame()
dct={"ByteTrack":"Bytetrack.json",  "ByteTrack+HMM":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder_H_0.07.json", "ByteTrack+Re-ID": "GR77_20200512_111314DBN_re_id_2.json"}# "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 
for idx, method in enumerate(dct.keys()):
    track_result= read_data(track_base+dct[method]) #### a refaire
    
    if method == "ByteTrack+HMM":
        df[method+"accuracy"], df[method+"recall"], df[method+"f1-score"], indices = precise_accuracy_track(label_track, track_result, basic_tracker=False)
    else:
        df[method+"accuracy"], df[method+"recall"], df[method+"f1-score"], indices = precise_accuracy_track(label_track, track_result, basic_tracker=True)
        
    print(method)
    df.index =indices
    fig_rec.add_trace(go.Scatter(x=df.index.values, y=df[method+"recall"].values,
                            mode='lines',
                            name='recall of '+method))
    fig_f1.add_trace(go.Scatter(x=df.index.values, y=df[method+"f1-score"].values, 
                            mode='lines',
                            name='f1-score of '+method))
    fig_acc.add_trace(go.Scatter(x=df.index.values, y=df[method+"accuracy"].values,
                            mode='lines',
                            name='accuracy of '+method))
    
    
    df =pd.DataFrame()
        

fig_f1.update_layout(
            xaxis_title=dict(text="frame number"),
            yaxis_title=dict(text= "f1-score"),
    font=dict(
             #   color="black",
                size=12
            ),
            )
fig_acc.update_layout(
            xaxis_title="frame number",
            yaxis_title="accuracy",
    font=dict(
             #   color="black",
                size=12
            ),
            )
fig_rec.update_layout(
            xaxis_title="frame number",
            yaxis_title="recall",
    font=dict(
             #   color="black",
                size=12
            ),
            )

fig_f1.show()
fig_rec.show()
fig_acc.show()

Plot IDF1 and clear metric mota

In [ ]:

dct={"hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder_H_0.07.json","Bytetrack2":"Bytetrack.json",   "Bytetrack_with_re_id": "GR77_20200512_111314DBN_re_id_1.json"}# "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}#  "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 

for key, value in dct.items():
    if ".h5" not in value:
        model_track = read_data(track_base+value)   
    """else:
        model_track = deeplabcut_model_track"""
    model_name=key
    
    print(model_name)
    model_summary=Evaluate_over_time(label_track, model_track, model_name)
    model_summary.to_excel(model_name+"_Tracking_result.xlsx",index_label="index")

    print("#################################################################DF")

    #model_summary_sf=Evaluate_single_frames_over_time(tracks, model_track, model_name)
    #model_summary_sf.to_excel("sf_"+model_name+"_Tracking_result.xlsx",index_label="index")


In [ ]:
#dct={"Bytetrack":"Bytetrack.json", "cross_input":"cross_input.json", "deepsort_smoothed_resut_without_max_thres":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut_training.h5"}# 
dct={"hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_result_feeder_H_0.07.json","Bytetrack2":"Bytetrack.json",   "Bytetrack_with_re_id": "GR77_20200512_111314DBN_re_id_1.json"}# "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}#  "deepsort":"deepsort_smoothed_resut_without_max_thres.json","deeplabcut":"deeplabcut.json", "cross_input":"cross_input.json"}# 

#"hmm_with_atq":"GR77_20200512_111314_with_atq_tracking_with_HMM_resut782.json"#
import pandas as pd
import copy
for idx, method in enumerate(dct.keys()):
    df_tmp = pd.read_excel(method+"_Tracking_result.xlsx")
    df_tmp.index = df_tmp['index']
    df_tmp = df_tmp.drop(['index'], axis=1)

    if idx>0:
        df_final = pd.merge(df_final, df_tmp, left_index=True, right_index=True)
    else:
        #print(df_tmp)
        df_first =copy.deepcopy(df_tmp)
        df_final =df_tmp
    
"""df_deepsort=pd.read_excel("deepsort_Tracking_result.xlsx")
df_deepsort.index = df_deepsort['index']

df_Bytetrack=pd.read_excel("Bytetrack_Tracking_result.xlsx")
df_Bytetrack.index = df_Bytetrack['index']

df_deeplabcut = pd.read_excel("deeplabcut_Tracking_result.xlsx")
df_deeplabcut.index = df_deeplabcut['index']

df_deepsort_smoothed = pd.read_excel("deepsort_smoothed_Tracking_result.xlsx")
df_deepsort_smoothed.index = df_deepsort_smoothed['index']

df_deepsort_smoothed_without_max =  pd.read_excel("deepsort_smoothed_resut_without_max_thres_Tracking_result.xlsx")
df_deepsort_smoothed_without_max.index =  df_deepsort_smoothed_without_max["index"]

df_cross_input =  pd.read_excel("cross_input_Tracking_result.xlsx")
df_cross_input.index =  df_cross_input["index"]

df_hmm=  pd.read_excel("hmm_with_atq_Tracking_result.xlsx")
df_hmm.index =  df_hmm["index"]



df = pd.merge(df_deepsort, df_deeplabcut, left_index=True, right_index=True)
df2 = pd.merge(df, df_deepsort_smoothed, left_index=True, right_index=True)
df3 = pd.merge(df2, df_deepsort_smoothed_without_max, left_index=True, right_index=True)
df4= pd.merge(df3, df_Bytetrack, left_index=True, right_index=True)
df5= pd.merge(df4, df_cross_input, left_index=True, right_index=True)
df6= pd.merge(df5, df_hmm, left_index=True, right_index=True)

df=df6
print(df.columns)
"""
import plotly.express as px
import plotly.graph_objects as go

df=  df_final
print(df.columns)

for col in df_first:# ["num_switches","mota", "idf1"]:# df_first: #['idp', 'mota','idr', idf1 ]:#
    col =col.split('_hmm_with_atq')[0]#because with the merge the name of the first key dictionnarry is added
    if "index" not in col:
        fig = go.Figure()
        print(col)
        """fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deepsort"].values,
                            mode='lines',
                            name=col+' of deepsort'))

        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deepsort_smoothed"].values,
                                mode='lines',
                                name=col+' of deepsort_smoothed'))"""

        """fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deeplabcut"].values,
                            mode='lines',
                            name=col+' of deeplabcut'))"""
        
        """fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_deepsort"].values,
                            mode='lines',
                            name=col+' of deepsort'))"""
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_Bytetrack2"].values,
                            mode='lines',
                            name=col+' of Bytetrack'))
        """fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_cross_input"].values,
                            mode='lines',
                            name=col+' of cross_input'))"""
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_hmm_with_atq"].values,
                            mode='lines',
                            name=col+' of hmm_with_atq'))
        fig.add_trace(go.Scatter(x=df.index.values, y=df[col+"_Bytetrack_with_re_id"].values,
                            mode='lines',
                            name=col+' of Bytetrack with Re-identification'))
        fig.update_layout(
            xaxis_title="frame number",
            yaxis_title=col,
            
            
    font=dict(
             #   color="black",
                size=15
            ),
                      
            )
        fig.show() 

# F1-score over time


In [ ]:
"""hmm_based= "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/re_id_vs_hmm_over_time/accuracy_over_nbr_of_visits_with_track_helping.csv"
re_id="/home/sophie/uncertain-identity-aware-tracking/Bytetrack/videos/re_id_vs_hmm_over_time/accuracy_Re_id_over_nbr_of_visits_with_track_helping.csv"

artificial_hmm = pd.read_csv(hmm_based)
artificial_re_id= pd.read_csv(re_id)

fig = go.Figure()
fig.add_trace(go.Scatter(x=artificial_hmm["nbr of visits"], y=artificial_hmm["f1"],
                    mode='lines',
                    name='HMM based tracker',
                    line_color="#0000ff"
                    ))
fig.add_trace(go.Scatter(x=artificial_re_id["nbr of visits"], y=artificial_re_id["f1"],
                    mode='lines',
                    name='Tracker with Re_id',
                    #line_color="#0000ff"
                    ))
fig.update_layout(
            xaxis_title="frame number",
            yaxis_title='f1 score',)"""

In [ ]:
hmm_based= "/home/sophie/uncertain-identity-aware-tracking/Bytetrack/results/accuracy_over_nbr_of_visits_with_track_helping14.csv"
re_id="/home/sophie/uncertain-identity-aware-tracking/Bytetrack/results/accuracy_Re_id_over_nbr_of_visits_with_track_helping14.csv"
artificial_hmm = pd.read_csv(hmm_based)
artificial_re_id= pd.read_csv(re_id)

all = []
for nbr_visit in np.unique(artificial_hmm["nbr of visits"]):
    mean = artificial_hmm[artificial_hmm["nbr of visits"]==nbr_visit].mean()["f1"]
    std = artificial_hmm[artificial_hmm["nbr of visits"]==nbr_visit].std()["f1"]
    all.append({"nbr of visits":nbr_visit,"mean f1":mean, "std f1":std})
artificial_hmm_c =pd.DataFrame(all)

all = []
for nbr_visit in np.unique(artificial_re_id["nbr of visits"]):
    mean = artificial_re_id[artificial_re_id["nbr of visits"]==nbr_visit].mean()["f1"]
    std = artificial_re_id[artificial_re_id["nbr of visits"]==nbr_visit].std()["f1"]
    all.append({"nbr of visits":nbr_visit,"mean f1":mean, "std f1":std})
artificial_re_id_c =pd.DataFrame(all)


fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=artificial_hmm_c["nbr of visits"], y=artificial_hmm_c["mean f1"],
                    mode='lines',
                    name='HMM based tracker',
                    line_color="#0000ff"
                    ))
"""fig.add_trace(go.Scatter(x=artificial_re_id["nbr of visits"], y=artificial_re_id["f1"],
                    mode='lines',
                    name='Tracker with Re_id',
                    #line_color="#0000ff"
                    ))"""
fig1.update_layout(
            xaxis_title="number of visits",
            yaxis_title='f1 score',)

df = artificial_hmm_c
df2 = artificial_re_id_c
fig = go.Figure([
    go.Scatter(
        name='Track with HMM',
        x=df['nbr of visits'],
        y=df['mean f1'],
        mode='lines',
        #line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=df['nbr of visits'],
        y=df['mean f1']+df['std f1'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=df['nbr of visits'],
        y=df['mean f1']-df['std f1'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ), 
    
    
    
    
     go.Scatter(
        name='Track with Re-id',
        x=df2['nbr of visits'],
        y=df2['mean f1'],
        mode='lines',
        #line=dict(color='rgb(31, 119, 180)'),
    ),
    go.Scatter(
        name='Upper Bound',
        x=df2['nbr of visits'],
        y=df2['mean f1']+df['std f1'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ),
    go.Scatter(
        name='Lower Bound',
        x=df2['nbr of visits'],
        y=df2['mean f1']-df['std f1'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ), 
    
    
])
fig.update_layout(
            xaxis_title="number of visits",
            yaxis_title='f1 score',
    title='F1 score over time with 25% of observations being random  (20 repetitions)',
    hovermode="x" ) 

fig.show()